<a href="https://colab.research.google.com/github/mgfrantz/CalTech-CTME-AramCo-2025/blob/main/notebooks/03_serve_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving and Testing Your Fine-tuned SQL Model

## What This Notebook Does

After fine-tuning our LoRA adapter in the previous notebook, we now need to:

1. **Serve the model** - Set up an inference server that can load both the base model and our LoRA adapter
2. **Test the adapter** - Validate that our fine-tuning improved SQL generation quality
3. **Compare performance** - See how the fine-tuned model performs vs. the base model

## Why Use vLLM for Serving?

**vLLM** is a high-performance inference server that offers:

- **Fast inference**: Optimized for serving large language models
- **LoRA support**: Can dynamically load different adapters without restarting
- **OpenAI-compatible API**: Easy integration with existing tools
- **Multi-adapter serving**: Serve multiple LoRA adapters simultaneously
- **Efficient memory usage**: Shares base model weights across adapters

## Architecture Overview

```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Base Model    │    │   LoRA Adapter   │    │   vLLM Server   │
│ (Llama-3.2-1B)  │ +  │  (SQL Tuning)    │ =  │  (Inference)    │
│                 │    │                  │    │                 │
└─────────────────┘    └──────────────────┘    └─────────────────┘
                                                        │
                                                        ▼
                                               ┌─────────────────┐
                                               │  OpenAI API     │
                                               │  Compatible     │
                                               │  Endpoint       │
                                               └─────────────────┘
```

## Step 1: Install Dependencies

We need several key packages for serving and testing:

- **vLLM**: High-performance inference server with LoRA support
- **litellm**: Unified API client for different LLM providers 
- **bitsandbytes**: Efficient quantization for memory optimization

**Installation time**: ~2-3 minutes

In [ ]:
!uv pip install -qqq vllm litellm bitsandbytes

## Step 2: Start the vLLM Server

**⚠️ IMPORTANT**: You need to run this command in a **separate terminal** because it starts a server process.

### Command Breakdown:

```bash
vllm serve NousResearch/Llama-3.2-1B \
    --max-model-len 2048 \
    --enable-lora \
    --max-lora-rank 32 \
    --lora-modules sql-lora=mgfrantz/NousResearch-Llama-3.2-1B-ctme-sql-demo
```

**Parameters explained**:

- `NousResearch/Llama-3.2-1B`: Base model to load
- `--max-model-len 2048`: Maximum sequence length (matches our training config)
- `--enable-lora`: Enable LoRA adapter support
- `--max-lora-rank 32`: Maximum LoRA rank (matches our training rank)
- `--lora-modules sql-lora=<adapter_path>`: Maps adapter name to HuggingFace model ID

### What Happens When You Run This:

1. **Downloads base model** (~2.5GB) - first time only
2. **Downloads LoRA adapter** (~60MB) - much smaller!
3. **Loads model into GPU memory** - requires ~4GB GPU RAM
4. **Starts HTTP server** on `http://localhost:8000`
5. **Ready for inference** - OpenAI-compatible API endpoints

### Expected Output:
```
INFO:     Started server process
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000
```

### Manual Steps:
1. Open a new terminal in Colab (click the terminal icon)
2. Run the vllm serve command above
3. Wait for "Application startup complete" message
4. Leave terminal running and return to this notebook

## Step 3: Download and Load Test Data

We'll use the same evaluation dataset we created during training to test our model's performance.
It's the same json in the chatml format.

In [ ]:
import os
import json
from rich import print

# Define file URLs and local filenames
urls = [
    "https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_evaluation_data.jsonl",
    "https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_training_data.jsonl",
]
filenames = [
    "chatml_evaluation_data.jsonl",
    "chatml_training_data.jsonl",
]

# Download files if they don't exist
for url, filename in zip(urls, filenames):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        !wget {url} -O {filename}
    else:
        print(f"{filename} already exists.")

In [ ]:
eval_conversations = []
with open('chatml_evaluation_data.jsonl', 'r') as f:
    for line in f:
        eval_conversations.append(json.loads(line))

## Step 4: Prepare a Test Case and Test the Fine-tuned Model

Extract the input messages (system + user) and expected answer from our test example.

**Test structure**:
- **Messages**: System prompt + user question with schema
- **Expected answer**: The correct SQL query our model should generate

Set up connection parameters for our vLLM server:

- **vllm_url**: Local server endpoint (default vLLM port is 8000)
- **model**: Specify our LoRA adapter name (`sql-lora` as defined in the serve command). We use `openai` to tell `litellm` that this is an openai-compatible server.

In [ ]:
from litellm import completion

In [ ]:
convo = eval_conversations[0].get('conversations')
messages = convo[:-1]
answer = convo[-1]

print("Messages:")
print(messages)

print("\n\nExpected answer:")
print(answer.get('content'))

In [ ]:
vllm_url = 'http://localhost:8000/v1/'
model = 'openai/sql-lora'

resp = completion(
    model,
    messages,
    base_url=vllm_url,
    api_key='not-needed',
    max_completion_tokens=128
)

print(resp.choices[0].message.content)